In [ ]:
!pip install accelerate
!pip install sentencepiece
!pip install datasets

In [ ]:
from transformers import MT5ForConditionalGeneration,AutoModelForSeq2SeqLM, T5Tokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer
import torch
import string
import pandas as pd
import sentencepiece
import accelerate
from torch.utils.data import Dataset, DataLoader, SequentialSampler, RandomSampler
from tqdm import tqdm
import re
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from tqdm import tqdm_notebook

In [ ]:
def read_data(file_path, num_lines=150000):
    with open(file_path, "r", encoding="utf-8") as f:
        return [next(f).strip() for _ in range(num_lines)]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
en_sents = read_data("/content/drive/MyDrive/new/data/en_sents.txt")
fr_sents = read_data("/content/drive/MyDrive/new/data/fr_sents.txt")
vi_sents = read_data("/content/drive/MyDrive/new/data/vi_sents.txt")

In [ ]:
# Create a DataFrame
df = pd.DataFrame({
    "en": en_sents,
    "fr": fr_sents,
    "vi": vi_sents
})
df.head()

,en,fr,vi
0,Please put the dustpan in the broom closet,Veuillez placer la pelle à poussière dans le p...,xin vui lòng đặt người quét rác trong tủ chổi
1,Be quiet for a moment.,silencieux un moment,im lặng một lát
2,Read this,lis ça,đọc này
3,Tom persuaded the store manager to give him ba...,Tom a persuadé le gérant du magasin de lui ren...,tom thuyết phục người quản lý cửa hàng trả lại...
4,Friendship consists of mutual understanding,L'amitié inclut la compréhension mutuelle,tình bạn bao gồm sự hiểu biết lẫn nhau


In [ ]:
def preprocess_text(text):
    # Loại bỏ dấu câu
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Chuyển thành chữ thường
    text = text.lower()
    # Loại bỏ khoảng trắng thừa
    text = text.strip()
    return text

# Tiền xử lý dữ liệu
en_sents = [preprocess_text(sent) for sent in en_sents]
fr_sents = [preprocess_text(sent) for sent in fr_sents]
vi_sents = [preprocess_text(sent) for sent in vi_sents]

In [ ]:
# Create a DataFrame
df = pd.DataFrame({
    "en": en_sents,
    "fr": fr_sents,
    "vi": vi_sents
})
df.head()

,en,fr,vi
0,please put the dustpan in the broom closet,veuillez placer la pelle à poussière dans le p...,xin vui lòng đặt người quét rác trong tủ chổi
1,be quiet for a moment,silencieux un moment,im lặng một lát
2,read this,lis ça,đọc này
3,tom persuaded the store manager to give him ba...,tom a persuadé le gérant du magasin de lui ren...,tom thuyết phục người quản lý cửa hàng trả lại...
4,friendship consists of mutual understanding,lamitié inclut la compréhension mutuelle,tình bạn bao gồm sự hiểu biết lẫn nhau


In [ ]:
# Tiền xử lý dữ liệu
source_texts = [f"<en> {en_sent}" for en_sent in en_sents] + [f"<fr> {fr_sent}" for fr_sent in fr_sents]
target_texts = vi_sents + vi_sents  # Gộp dữ liệu tiếng Việt lại

In [ ]:
df = pd.DataFrame({
    "input_text": source_texts,
    "target_text": target_texts,
})
df.sample(6)

,input_text,target_text
213076,<fr> rentrez à la maison en toute sécurité,về nhà an toàn
178195,<fr> nous découvrirons la vérité,chúng ta sẽ tìm ra sự thật
205138,<fr> veuxtu dîner ici avec moi,bạn sẽ ăn tối ở đây với tôi
165955,<fr> jai dit à tom tout ce que je savais,tôi đã nói với tom tất cả những gì tôi biết
84271,<en> ill figure it out,tôi sẽ tìm ra nó
32900,<en> this man is chinese,người đàn ông này là người trung quốc


In [ ]:
# Xáo trộn DataFrame
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Chia thành train, eval, và test theo tỉ lệ mong muốn (ví dụ: 80-10-10)
train_df, eval_df = train_test_split(df_shuffled, test_size=0.1, random_state=42)

# In kích thước của từng tập dữ liệu
print("Train set size:", len(train_df))
print("Eval set size:", len(eval_df))

Train set size: 270000
Eval set size: 30000


In [ ]:
train_df

,input_text,target_text
299465,<fr> elle a mordu plus quelle ne pouvait mâche...,cô cắn nhiều hơn những gì cô có thể nhai và cười
81922,<en> im going to take my car,tôi sẽ đi xe của tôi
143673,<en> they came up with a plan after a long dis...,họ đã đưa ra một kế hoạch sau một cuộc thảo lu...
208449,<fr> dick ma passé la photo,tinh ranh truyền cho tôi bức ảnh
293179,<fr> leurs parents sont plus âgés que nous,cha mẹ của họ lớn tuổi hơn chúng ta
...,...,...
119879,<en> help me lift the package,giúp tôi nâng gói
259178,<en> is there anything you want me to bring,có bất cứ điều gì bạn muốn tôi mang lại
131932,<fr> je narrive pas à croire à quel point tu e...,tôi không thể tin rằng bạn đẹp như thế nào
146867,<en> go back to work,trở lại làm việc


In [ ]:
eval_df

,input_text,target_text
4941,<fr> sa gentillesse ma donné une boule dans la...,lòng tốt của cô ấy cho tôi một khối u trong cổ...
51775,<en> i thought youd be too busy to see me,tôi nghĩ bạn sẽ quá bận để gặp tôi
115253,<fr> vous devez venir immédiatement,bạn cần phải đến ngay lập tức
299321,<en> tom took a beer out of the fridge and han...,tom lấy bia ra khỏi tủ lạnh và đưa nó cho mary
173570,<en> this childs mother is an announcer,mẹ của đứa trẻ này là một phát thanh viên
...,...,...
199500,<en> tom and mary laughed at each other,tom và mary cười với nhau
244038,<fr> elle brûlait de jalousie,cô ấy đang bùng cháy vì ghen tị
79446,<en> whats the forecast,dự báo thời tiết nói gì
276390,<en> i think ill wait here,tôi nghĩ tôi sẽ đợi ở đây


In [ ]:
train_df = train_df.reset_index(drop=True)
eval_df = eval_df.reset_index(drop=True)

In [ ]:

model_repo = 'google/mt5-small'
model_path= '/content/drive/MyDrive/new/results/model/mt5_translation.pt'
max_seq_len = 40

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_repo)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_repo)
model = model.cuda()

In [ ]:
special_tokens_dict = {'additional_special_tokens': ['<en>', '<fr>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(250102, 512)

In [ ]:
model.config.max_length=40

In [ ]:

len(tokenizer.vocab)

250102

In [ ]:
tokenizer.all_special_tokens

['</s>', '<unk>', '<pad>', '<en>', '<fr>']

In [ ]:
def encode(examples):
    inputs = tokenizer(examples['input_text'], truncation=True, max_length=max_seq_len, padding='max_length', return_tensors='pt')
    targets = tokenizer(examples['target_text'], truncation=True, max_length=max_seq_len, padding='max_length', return_tensors='pt')
    inputs['labels'] = targets['input_ids']
    return inputs

In [ ]:
# Encode dữ liệu
train_dataset = train_df.apply(encode, axis=1)
eval_dataset = eval_df.apply(encode, axis=1)

In [ ]:
sample = train_dataset.iloc[0]

print(f"input_text: {train_df.iloc[0]['input_text']}")
print(f"target_text: {train_df.iloc[0]['target_text']}")
print(f"input_ids: {sample['input_ids']}")
print(f"attention_mask: {sample['attention_mask']}")
print(f"labels: {sample['labels']}")

input_text: <fr> elle a mordu plus quelle ne pouvait mâcher et a ri
target_text: cô cắn nhiều hơn những gì cô có thể nhai và cười
input_ids: tensor([[250101,  15679,    259,    262,  81873,    273,   1245,    259,  16141,
            448,   2001,  65040,  60730,   5625,    383,    259,    262,   1418,
              1,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0]])
attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
labels: tensor([[ 6740,   317,  6701,   677,  1990,   382,  2238,   259,   272,  1992,
           259,   318,  1135,  6740,   885,   394,   924,   677,   741,   259,
           793,   317, 25341,     1,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])


In [ ]:
# Tạo DataLoaders cho tập huấn luyện và đánh giá
batch_size = 16

train_dataloader = DataLoader(
            train_dataset,  # Dữ liệu huấn luyện
            sampler = RandomSampler(train_dataset), # Chọn mẫu ngẫu nhiên để huấn luyện
            batch_size = batch_size # Huấn luyện với batch_size này
        )

validation_dataloader = DataLoader(
            eval_dataset, # Dữ liệu đánh giá
            sampler = SequentialSampler(eval_dataset), # Chọn mẫu tuần tự để đánh giá
            batch_size = batch_size # Đánh giá với batch_size này
        )

In [ ]:
num_epochs = 5
total_steps = len(train_dataset) * num_epochs
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
# Huấn luyện mô hình
for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):
        input_ids = batch['input_ids'].squeeze().to(device)
        attention_mask = batch['attention_mask'].squeeze().to(device)
        labels = batch['labels'].squeeze().to(device)

        model.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_train_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        # In ra loss sau mỗi 100 bước
        if step % 100 == 0 and not step == 0:
            print(f'  Batch {step}  of  {len(train_dataloader)}.    Loss: {loss.item()}')

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f'Average training loss: {avg_train_loss}')

Epoch 1/5
----------
  Batch 100  of  16875.    Loss: 41.421974182128906
  Batch 200  of  16875.    Loss: 42.56519317626953
  Batch 300  of  16875.    Loss: 37.97869110107422
  Batch 400  of  16875.    Loss: 34.817527770996094
  Batch 500  of  16875.    Loss: 33.18784713745117
  Batch 600  of  16875.    Loss: 30.99874496459961
  Batch 700  of  16875.    Loss: 28.66680908203125
  Batch 800  of  16875.    Loss: 27.971582412719727
  Batch 900  of  16875.    Loss: 24.539335250854492
  Batch 1000  of  16875.    Loss: 23.766845703125
  Batch 1100  of  16875.    Loss: 22.85208511352539
  Batch 1200  of  16875.    Loss: 21.820247650146484
  Batch 1300  of  16875.    Loss: 19.88681411743164
  Batch 1400  of  16875.    Loss: 21.37656021118164
  Batch 1500  of  16875.    Loss: 18.614505767822266
  Batch 1600  of  16875.    Loss: 16.65066146850586
  Batch 1700  of  16875.    Loss: 15.956418991088867
  Batch 1800  of  16875.    Loss: 15.463096618652344
  Batch 1900  of  16875.    Loss: 15.209953308

In [ ]:
model.save_pretrained('/content/drive/MyDrive/new/results/model3')
tokenizer.save_pretrained('/content/drive/MyDrive/new/results/token')

('/content/drive/MyDrive/new/results/token/tokenizer_config.json',
 '/content/drive/MyDrive/new/results/token/special_tokens_map.json',
 '/content/drive/MyDrive/new/results/token/spiece.model',
 '/content/drive/MyDrive/new/results/token/added_tokens.json',
 '/content/drive/MyDrive/new/results/token/tokenizer.json')

In [ ]:
def translate(text, src_lang):
    # Đưa mô hình và dữ liệu vào cùng một thiết bị
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Mã hóa văn bản đầu vào
    encoded_text = tokenizer(f"<{src_lang}> {text}", return_tensors='pt').to(device)

    # Dịch văn bản
    translated = model.generate(**encoded_text)

    # Giải mã văn bản đã dịch
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)

    return translated_text